# DFTTK needs the anaconda environment.
If not run under Jupyter notebook environment,
One can download anaconda from https://www.anaconda.com/ for Mac and Windows, 
and https://docs.anaconda.com/anaconda/install/linux/ for linux

# install dfttk 
1. click file->new->Terminal by this server. Otherwise, one can open an Linux shell, 
or an Anaconda Powershell Prompt (run as Adminstrator)
2. To find your home folder which will be used later, type
```
pwd
```
3. Install gcc which is needed by some dependencies, type
```
conda install gcc
```
4. Clone the latest version of DFTTK from github
```
git clone https://github.com/yiwang62/dfttk
```
5. Install the develop version of dfttk

```
cd dfttk
pip install -e .
```

# config the environment of pymatgen which is the engine through it to run VASP

Here we tell pymatgen where the pseudopotentials are saved.

```
cd ~/dfttk
mkdir vasp_psp
cd vasp_psp/
mkdir POT_GGA_PAW_PBE_52/  POT_GGA_US_PW91/  POT_LDA_PAW_52/  POT_LDA_US/
mkdir POT_GGA_PAW_PBE/  POT_GGA_PAW_PBE_54/  POT_LDA_PAW/      POT_LDA_PAW_54/
```
You noted that these are paths for different VASP pseudopotentials to copy to

# Setup DFTTK run enviroment

Ask the MongoDB system manager for two files: one named "db.json", another named "my_launchpad.yaml". These two files can be generated by running a code under the scripts folder by your system manager or you.

```
python dfttk/scripts/mongodb_user.py 
```

the "db.json" is a dict like

```
{
    "database": "userid-results",
    "collection": "tasks",
    "admin_user": "userid",
    "admin_password": "pass1",
    "readonly_user": "userid-ro",
    "readonly_password": "pass2",
    "host": "146.186.149.69",
    "port": 27018,
    "aliases": {}
}
```

and the "my_launchpad.yaml" file is used by "FireWorks" for DFT job managements through MongoDB , templated as follows

```
host: 146.186.149.69
name: userid-fws
password: pass3
port: 27018
ssl_ca_file: null
strm_lvl: INFO
user_indices: []
username: userid
wf_user_indices: []
```


# Config dfttk config by run

1. make a folder named "config" and save the "db.json" and "my_launchpad.yaml" files to that folder
```
cd ~/dfttk
mkdir config
```

2. Finlize dfttk setup by run
```
dfttk config -a -M cori-knl -psp vasp_psp
```

where "cori-knl" is one of supercomputers for them the batch job queue system have been predefined.

After this step you should modify several files under the config folder

```
vi config/my_qadapter.yaml 

_fw_name: CommonAdapter
_fw_q_type: SLURM
rocket_launch: rlaunch -c /home/jovyan/dfttk/config rapidfire
nodes: 1
ntasks: 16
walltime: '48:00:00'
queue: regular
account: m891
job_name: dfttk
pre_rocket: module load vasp/5.4.4-knl
post_rocket: ''
logdir: /home/jovyan/dfttk/logs
constraint: knl,quad,cache
mem: 64gb
vasp_cmd: srun -n 16 --cpu_bind=cores vasp_std

```

```
vi config/my_fworker.yaml

name: cori-knl
category: ''
query: '{}'
env:
  db_file: /home/jovyan/dfttk/config/db.json
  vasp_cmd: srun -n 16 --cpu_bind=cores vasp_std
  scratch_dir: 'null'
  incar_update: {}
  
```
Note that a environment file, ".pmgrc.yaml ", for pymatgen is also crearted in your home folder
```
vi ~/.pmgrc.yaml 

PMG_DEFAULT_FUNCTIONAL: PBE
PMG_VASP_PSP_DIR: /home/jovyan/dfttk/vasp_psp
PMG_MAPI_KEY:
```


In [ ]:
#set enviromental variable by shell or jupyter notebook by

import os
os.environ["FW_CONFIG_FILE"] = "/home/jovyan/dfttk/config/FW_config.yaml" 

# or by shell command "export FW_CONFIG_FILE=/home/jovyan/dfttk/config/FW_config.yaml"
# Note "/home/jovyan/" is your home folder, yours is different from this one


# go to your home folder

```
cd ~/
mkdir AlNi
cd AlNi
```


# prepare the "POSCAR.AlNi" file with the following data
```
Al1 Ni1
1.0
2.887902 0.000000 0.000000
0.000000 2.887902 0.000000
0.000000 0.000000 2.887902
Al Ni
1 1
direct
0.000000 0.000000 0.000000 Al
0.500000 0.500000 0.500000 Ni
```

# prepare the "SETTINGS.yaml" file with the following data
```
phonon : True
phonon_supercell_matrix : [[0, 2, 2], [2, 0, 2], [2, 2, 0]]
```

In [ ]:
# go to the "AlNi" folder
import os
os.chdir("/home/jovyan/AlNi") 
# submoit your dfttk job by
!dfttk run -f POSCAR.AlNi -l #if in the terminal windos, simply run "dfttk run -f POSCAR.AlNi -l"

# login in supercomputer and run there `qlaunch singleshot'


In [ ]:
# you can check your job status by
os.chdir("/home/jovyan/") # go back to home

!lpad get_wflows

In [ ]:
# run

!lpad -h for all options

In [ ]:
# you can find you finished dfttk calculations by

!dfttk thfind

In [ ]:
#you can perform result analysis by
!dfttk thfind -tag 0362ea63-5553-4156-8378-d0c4f80fed4f -get -plot DFTTK

In [ ]:
# Check the output

# distribute DFT jobs to multi supercomputers
run example using terminal

```
cd ~/
```

```
mkdir AlNi3
cd AlNi3

vi POSCAR.AlNi3

Al1 Ni3
1.0
3.561084 0.000000 0.000000
0.000000 3.561084 0.000000
0.000000 0.000000 3.561084
Al Ni
1 3
direct
0.000000 0.000000 0.000000 Al
0.500000 0.500000 0.000000 Ni
0.500000 0.000000 0.500000 Ni
0.000000 0.500000 0.500000 Ni

vi SETTINGS.yaml

phonon : True
phonon_supercell_matrix : [[2, 0, 0], [0, 2, 0], [0, 0, 2]]

```

# submit jobs

```
dfttk run -wf crosscom -f POSCAR.AlNi3 -l
```

In [ ]:
# check your DFTTK job status
!lpad get_wflows

# run VASP job

if have followed the above steps and installed DFTTK in a supercomputer with VASP installed, then you now go to your $SCRATCH drive, create a folder where you want to save the imtermidate output from DFTTK, followed by launching VASP jobs by

```
qlaunch singleshot #single batch job
```

or 

```
qlaunch rapidfire #submit many batch jobs and you have to kill it by 'ctrl-C' to teminate qlaunch. Otherwise, you will have thousands batch job.
```

# install DFTTK in multiple computer

say one is your PC/Mac that does not have VASP installed and another is supercomputer having VASP installed, then you need to install DFTTK in both computer, i.e.

1. submit your DFTTK jobs from either your PC/Mac
2. qlaunch your DFT VASP jobs from the computer where VASP is installed

## note that when submit the DFTTK jobs, it only means submitting the jobs to the MongoDB database, laterly, any supercomputers that have access to the MongoDB database can run the VASP jobs by picking

Is this amazing?

# Make your own MongoDB server

https://www.dfttk.org/en/master/VM_MongoDB.html

The typical costs is around a couple of hundreds dollars a year for a storage of 100 GB and four cores.

#Some useful MongoDB commands

```
db.getUsers()
show users
show dbs

```
